## Benchmark Question-Answer Generation

This notebook demonstrates how we can generate a set of Questions and Answers based on chunks from a database. Documents chunked before insertion to a database and saved to `.csv` for MSFT transcripts sample. It is beneficial to generate questions and answers at this stage, as doing a search from a database has an added cost. Randomly picking chunks for specified filter parameters ensures diversity of the questions and answers. 

### SET UP AND CONFIGURATION

Load Environment File

In [1]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "../../llm.env" # change to your own .env file name
config = dotenv_values(env_name)

Read Chunks from csv (see step2 notebook from preprocessing subdirectory)

In [8]:
import numpy as np
import pandas as pd
df = pd.read_csv('AnalyzedPDF/ChunksEmbedding.csv')

In [4]:
df[df['Quarter']==2]

,Id,Ticker,Year,Quarter,Chunk,PageNumber,LineNumber,Embedding
113,114,MSFT,23,2,Microsoft FY23 Second Quarter Earnings Confere...,1,1,"[-0.022043932229280472, -0.023832328617572784,..."
114,115,MSFT,23,2,"On the Microsoft Investor Relations website, y...",1,9,"[-0.023697681725025177, -0.005627374164760113,..."
115,116,MSFT,23,2,GAAP. They are included as additional clarifyi...,1,17,"[-0.012550131417810917, -0.0020706052891910076..."
116,117,MSFT,23,2,"same in constant currency, we will refer to th...",1,25,"[-0.017685849219560623, -0.02943631075322628, ..."
117,118,MSFT,23,2,"predictions, projections, or other statements ...",2,6,"[-0.00904887169599533, -0.01967030204832554, -..."
...,...,...,...,...,...,...,...,...
216,217,MSFT,23,2,"BRETT IVERSEN: Thanks, Brad. Joe, we have time...",31,13,"[0.00118646037299186, -0.040140919387340546, 0..."
217,218,MSFT,23,2,the coming quarters? Thank you. SATYA NADELLA:...,31,21,"[0.009347211569547653, -0.010082229971885681, ..."
218,219,MSFT,23,2,"going to be an Al app. That's, I think, the be...",32,1,"[0.005495850928127766, -0.0035756349097937346,..."
219,220,MSFT,23,2,"Sometimes, you will have ISVs who are differen...",32,9,"[-0.0043395268730819225, -0.02853129617869854,..."


In [5]:
df['Chunk'].iloc[11]

"Cosmos DB now supports PostgreSQL, making Azure the first cloud provider to offer a database service that supports both relational and NoSQL workloads. And, in Al, we are turning the world's most advanced models into platforms for customers. Earlier this month, we brought the power of Dall-E to Azure OpenAI service, helping customers like Mattel apply the breakthrough image generation model to commercial use cases for the first time. And Azure Machine Learning provides industry leading MLOps, helping organizations like 3M deploy, manage, and govern models. "

#### Prompt Template
##### Write a Prompt Tempalte. The prompt template should include all filter keys, so they can be referenced and input.

In [21]:
template = """
        You are a question answer generator.You are given two chunks of text, a ticker e.g. MSFT, Quarter, Year, as input. You will generate 10 relevant questions and answers grounded in the text. The question should be such that the answer is available in both or either of the chunks. They should try to cover most of the information in the text.
               
        An example output for this example is: 

        Question: For {ticker} FY{year} Q{quarter}, what is the <question goes here>?
        Answer: example answer goes here

        Based on ticker, quarter, year, the question can be phrased in different ways e.g. MSFT FY23 Q1, MSFT FY2023 1st quarter, e.t.c.
        In case the text question is not relevant, please skip the question and answer pair.
        input_text1: 
        {chunk_text1}
        input_text2:
        {chunk_text2}
        ticker: {ticker}
        quarter: {quarter}
        year: {year}
        """

#### Randomly pick filter parameters:

Add Filter Parameters to randomly pick them for extracting context (chunks). This will help diversify generating questions and answers. 
In the MSFT Financial Transcripts use-case, the Ticker name is MSFT, but you can easily add other ticker labels for a larger Financial dataset. `Year`,`Quarter`, and `Id` are the key parameters used in this use-case. In this notebook, we are using two random chunks (chunk ids) from specified filter keys (year, quarter).

###### TODO: Add Tools (an updated version of function calls) once it is available for working with newer models. Function Calls are currently deprecated for gpt models with versions beyond 07-01-2023.

In [9]:
Ticker = np.random.choice(df['Ticker'].unique())
Year = np.random.choice(df[df['Ticker']==Ticker]['Year'].unique())
Quarter = np.random.choice(df[(df['Ticker']==Ticker) & (df['Year']==Year)]['Quarter'].unique())
Id = np.random.choice(df[(df['Quarter']==Quarter) & (df['Year']==Year)]['Id'].unique())
Id2 = np.random.choice(df[(df['Quarter']==Quarter) & (df['Year']==Year) & (df['Id']!=Id)]['Id'].unique())

In [10]:
Ticker

'MSFT'

#### Generate Questions (using Azure OpenAI only)

In [22]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = config["OPENAI_API_KEY"],  
  api_version = config["OPENAI_API_VERSION"],
  azure_endpoint = config["OPENAI_API_BASE"]
)

response = client.chat.completions.create(
    model="gpt-35-turbo", # model = "deployment_name".
    messages=[
        {"role": "system", "content":"You are a generator of questions and answers for the given text." },
        {"role": "user", "content": template.format(chunk_text1=df['Chunk'].iloc[Id], chunk_text2=df['Chunk'].iloc[Id2], 
                                                    ticker=Ticker, year=str(Year), quarter=str(Quarter))}
    ]
)

#print(response)
# print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

#TODO: Add cells showing adding tools to the chat completion. It is an update to functionc calling feature. Function calling feature is not available in the current version of the API.OpenAI version > 1.0.0.

1. What was the percentage increase in server products and cloud services revenue for MSFT in Q1 FY23?
Answer: The server products and cloud services revenue increased by 22% in constant currency for MSFT in Q1 FY23.

2. How much did Azure and other cloud services revenue grow in constant currency for MSFT in Q1 FY23?
Answer: Azure and other cloud services revenue grew 35% in constant currency for MSFT in Q1 FY23.

3. Did Azure consumption growth meet expectations for MSFT in Q1 FY23?
Answer: No, Azure consumption growth was about a point lower than expected for MSFT in Q1 FY23.

4. What was the impact on the enterprise mobility and security installed base for MSFT in Q1 FY23?
Answer: The enterprise mobility and security installed base grew by 18% to over 232 million seats for MSFT in Q1 FY23.

5. How did the revenue for MSFT's on-premises server business perform in Q1 FY23?
Answer: The revenue was flat and increased 4% for MSFT's on-premises server business in Q1 FY23.

6. What was th

In [12]:
df[(df['Quarter']==Quarter) & (df['Year']==Year) & (df['Id']==Id)].Chunk.values[0]

"little bit of help there. We did not see as big of that. As I said, it's over 800 for the year. Some of that was in Q1, but the majority of it will be in Q2 through 4. And I think if you want to think about it as somewhere, 250-ish a quarter. It's not exact, but that would be a decent assumption for the remainder of the year. MARK MURPHY, JP Morgan: Thank you. BRETT IVERSEN: Thanks, Mark. Jessi, next question, please. (Operator Direction.) KARL KEIRSTEAD, UBS: Okay, great. Amy, I've love to just ask you around margins. Clearly, you're experiencing a little bit more sales mix, given the "

In [19]:
import re
_pattern = re.compile(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s')
retrieved_sentences=_pattern.split(df[(df['Quarter']==Quarter) & (df['Year']==Year) & (df['Id']==Id)].Chunk.values[0])
retrieved_sentences

['little bit of help there.',
 'We did not see as big of that.',
 "As I said, it's over 800 for the year.",
 'Some of that was in Q1, but the majority of it will be in Q2 through 4.',
 'And I think if you want to think about it as somewhere, 250-ish a quarter.',
 "It's not exact, but that would be a decent assumption for the remainder of the year.",
 'MARK MURPHY, JP Morgan: Thank you.',
 'BRETT IVERSEN: Thanks, Mark.',
 'Jessi, next question, please.',
 '(Operator Direction.) KARL KEIRSTEAD, UBS: Okay, great.',
 "Amy, I've love to just ask you around margins.",
 "Clearly, you're experiencing a little bit more sales mix, given the "]

In [20]:
retrieved_sentences2=_pattern.split(df[(df['Quarter']==Quarter) & (df['Year']==Year) & (df['Id']==Id2)].Chunk.values[0])
retrieved_sentences2

['year-over-year.',
 'Excluding the impact of the latest change in accounting estimate, gross margin percentage decreased slightly driven by sales mix shift to cloud offerings.',
 'Operating expense increased 13 percent and 16 percent in constant currency, and operating income increased 10 percent and 19 percent in constant currency, including 4 points due to the latest change in accounting estimate.',
 'Next, the Intelligent Cloud segment.',
 'Revenue was $20.3 billion, increasing 20 percent and 26 percent in constant currency, in line with expectations.',
 'Overall, server products and cloud services revenue increased 22 percent and 28 percent in constant currency.',
 'Azure and other cloud services ']

In [ ]:
response.choices[0].message.content